# Assignment 2

#### First Name: Jaime Leonardo
#### Last Name: Sánchez Salazar


## 1. Load Data from JSON

In [0]:
# Make sure to upload the corona_tweet_new.json file
df_twitter = spark.read.json("/FileStore/tables/corona_tweet_new.json")

In [0]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- user: struct (nullable = true)
 |    |-- contributors_enabled: boolean (nullable = true)
 |    |-- created_at: string (nullable = true)
 |    |-- default_profile: boolean (nullable = true)
 |    |-- default_profile_image: boolean (nullable = true)
 |    |-- description: string (nullable = true)
 |    |-- favourites_count: long (nullable = true)
 |    |-- follow_request_sent: string (nullable = true)
 |    |-- followers_count: long (nullable = true)
 |    |-- following: string (nullable = true)
 |    |-- frie

In [0]:
### From the user nestec col select the following cols only id_str,followers_count,friends_count and created at 
# (2 points)
from pyspark.sql.functions import col 
df_twitter=df_twitter.select(col("created_at"),
                             col("favorite_count"),
                             col("hashtags"),
                             col("id"),
                             col("in_reply_to_status_id"),
                             col("in_reply_to_user_id_str"),
                             col("location"),
                             col("reply_count"),
                             col("retweet_count"),
                             col("source"),
                             col("user.id_str"),
                             col("user.followers_count"),
                             col("user.friends_count"),
                             col("user.created_at").alias("user_created_at")
                            )

In [0]:
df_twitter.printSchema()

root
 |-- created_at: string (nullable = true)
 |-- favorite_count: long (nullable = true)
 |-- hashtags: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- id: string (nullable = true)
 |-- in_reply_to_status_id: string (nullable = true)
 |-- in_reply_to_user_id_str: string (nullable = true)
 |-- location: string (nullable = true)
 |-- reply_count: long (nullable = true)
 |-- retweet_count: long (nullable = true)
 |-- source: string (nullable = true)
 |-- id_str: string (nullable = true)
 |-- followers_count: long (nullable = true)
 |-- friends_count: long (nullable = true)
 |-- user_created_at: string (nullable = true)



In [0]:
# Print the total count of number of records in df_twitter(1 point)
df_twitter.count()

Out[5]: 15894

In [0]:
# Extract the source lable from source col by droping the anchor tab and save it as another col named extracted_source
# for example <a href="https://mobile.twitter.com" rel="nofollow">Twitter Web App</a> => Twitter Web App
# you can use "<a [^>]+>([^<]+)" as regualr expresion and the group would be 1 for this regular expression.
#(4 points)
from pyspark.sql.functions import regexp_extract, col

df_twitter=df_twitter.withColumn("extracted_source", regexp_extract(col("source"), '<a [^>]+>([^<]+)', 1))
df_twitter.select(col('extracted_source'),col('source')).show()

+-------------------+--------------------+
|   extracted_source|              source|
+-------------------+--------------------+
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter Web Client|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
|Twitter for Android|<a href="http://t...|
|    Twitter Web App|<a href="https://...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
| Twitter for iPhone|<a href="http://t...|
|Twitter for Android|<a href="http://t...|
+----------

In [0]:
# Convert the DataFrame into RDD
rdd_twitter=df_twitter.rdd.map(tuple)

In [0]:
# Create a temporay table in memory with name as twitter (1 point)
df_twitter.createOrReplaceTempView("twitter")

## 2. Analyze Data

#### You will be writing code to find the answer to the questions listed below using Just RDD, Using spark SQL 

- Analyze using RDD 
- Analyze using Dataframe without temp table 
- Analyze using spark.sql with temp table


#### 2.1 Get total number of unique users (1 point for each type)

In [0]:
# Using RDD
user_index = df_twitter.columns.index('id_str')
unique_users_rdd = rdd_twitter.map(lambda row: row[user_index]).distinct()
total_unique_users = unique_users_rdd.count()
print(f"Total number of unique users: {total_unique_users}")

Total number of unique users: 14094


In [0]:
# Using DataFrame
total_unique_users_df = df_twitter.select(col('id_str')).distinct().count()
print(f"Total number of unique users: {total_unique_users_df}")

Total number of unique users: 14094


In [0]:
# Using spark.sql and the temporay table.
spark.sql("SELECT COUNT(DISTINCT id_str) FROM twitter").show()

+----------------------+
|count(DISTINCT id_str)|
+----------------------+
|                 14094|
+----------------------+



#### 2.2 Get count of user who have more than 1 tweet in the data (2 points)

In [0]:
# Using RDD
user_index = df_twitter.columns.index('id_str')
tweet_count = rdd_twitter.map(lambda row: row[user_index]).map(lambda user: (user, 1)).reduceByKey(lambda x, y: x + y)
count = tweet_count.filter(lambda x: x[1] > 1).count()
print('Number of users with more than 1 tweet in the data:', count)


Number of users with more than 1 tweet in the data: 1016


In [0]:
# Using DataFrame
tweet_count = df_twitter.groupby("id_str").count().filter("count > 1").count()
print('Number of users with more than 1 tweet in the data:', tweet_count)

Number of users with more than 1 tweet in the data: 1016


In [0]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(*) FROM (SELECT COUNT(id_str) AS tweet_count FROM twitter GROUP BY id_str) WHERE tweet_count > 1').show()

+--------+
|count(1)|
+--------+
|    1016|
+--------+



#### 2.3 Get total number unique extracted_source (1 point each)

In [0]:
# Using RDD
extracted_source_index = df_twitter.columns.index('extracted_source')
extracted_source_rdd = rdd_twitter.map(lambda row: row[extracted_source_index]).distinct().count()
print('Number of unique extracted_source :', extracted_source_rdd)

Number of unique extracted_source : 133


In [0]:
# Using DataFrame
source_count = df_twitter.select("extracted_source").distinct().count()
print('Number of unique extracted_source :', source_count)

Number of unique extracted_source : 133


In [0]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(DISTINCT extracted_source) FROM twitter').show()

+--------------------------------+
|count(DISTINCT extracted_source)|
+--------------------------------+
|                             133|
+--------------------------------+



#### 2.4 Get top 5 most used extracted_source

In [0]:
# Using RDD (5 points)
extracted_source_top = (
    rdd_twitter
    .map(lambda row: (row[extracted_source_index], 1)) 
    .reduceByKey(lambda x, y: x + y)  
    .sortBy(lambda x: x[1], ascending=False) 
)
extracted_source_top.take(5)

Out[18]: [('Twitter for Android', 6262),
 ('Twitter for iPhone', 5698),
 ('Twitter Web App', 2878),
 ('Twitter for iPad', 428),
 ('Twitter Web Client', 136)]

In [0]:
# Using DataFrame (2 points)
df_twitter.groupby("extracted_source").count().sort("count", ascending = False).show(5)

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+
only showing top 5 rows



In [0]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('SELECT extracted_source, COUNT(extracted_source) as count FROM twitter GROUP BY extracted_source ORDER BY count DESC LIMIT 5').show()

+-------------------+-----+
|   extracted_source|count|
+-------------------+-----+
|Twitter for Android| 6262|
| Twitter for iPhone| 5698|
|    Twitter Web App| 2878|
|   Twitter for iPad|  428|
| Twitter Web Client|  136|
+-------------------+-----+



#### 2.5 Get count of distinct hastags used ( 5 point each) 

In [0]:
# Using RDD
hashtags_index = df_twitter.columns.index('hashtags')
hashtags_count = rdd_twitter.flatMap(lambda row: row[hashtags_index])
hashtags_count.filter(lambda hashtags: hashtags is not None and len(hashtags) > 0).distinct().count()

Out[21]: 1215

In [0]:
# Using DataFrame
from pyspark.sql.functions import explode

hashtags_count  = df_twitter.select(explode(col('hashtags'))).distinct().count()
print('The number of distinct hastags used is:', hashtags_count)

The number of distinct hastags used is: 1215


In [0]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(*) FROM (SELECT DISTINCT EXPLODE(hashtags) FROM twitter)').show()

+--------+
|count(1)|
+--------+
|    1215|
+--------+



#### 2.6 Get top 5 hashtags

In [0]:
# Using RDD (4 points)
hashtags_index = df_twitter.columns.index('hashtags')
hashtags_top = rdd_twitter.flatMap(lambda row: row[hashtags_index])
hashtags_top.map(lambda hashtags: (hashtags, 1)).reduceByKey(lambda x, y: x + y).sortBy(lambda x: x[1], ascending=False).take(5)

Out[24]: [('طبق_القدرات_للثانويه_ياريس', 385),
 ('Corona', 319),
 ('OilPrice', 251),
 ('COVID19', 125),
 ('corona', 123)]

In [0]:
# Using DataFrame (2 points)
df_twitter.select(explode(col('hashtags'))).groupby("col").count().sort("count", ascending = False).show(5)

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+
only showing top 5 rows



In [0]:
# Using spark.sql and the temporay table. (2 points)
spark.sql('SELECT col, COUNT(col) as count FROM (SELECT EXPLODE(hashtags) FROM twitter) GROUP BY col ORDER BY count DESC LIMIT 5').show()

+--------------------+-----+
|                 col|count|
+--------------------+-----+
|طبق_القدرات_للثان...|  385|
|              Corona|  319|
|            OilPrice|  251|
|             COVID19|  125|
|              corona|  123|
+--------------------+-----+



#### 2.6 Get total number of tweets which are retweeted more than 100 times

In [0]:
# Using RDD
retweet_count_index = df_twitter.columns.index('retweet_count')
rdd_twitter.filter(lambda row: row[retweet_count_index] > 100).count()

Out[27]: 15753

In [0]:
# Using DataFrame
count_100 = df_twitter.filter("retweet_count > 100").count()
print('The number of tweets which are retweeted more than 100 times is:', count_100)

The number of tweets which are retweeted more than 100 times is: 15753


In [0]:
# Using spark.sql and the temporay table.
spark.sql('SELECT COUNT(id) FROM twitter WHERE retweet_count > 100').show()

+---------+
|count(id)|
+---------+
|    15753|
+---------+



#### 2.8 Get top 3 most retweeted tweets per country (8 points)

In [0]:
# Using RDD
location_index = df_twitter.columns.index('location')
id_index = df_twitter.columns.index('id')
retweet_count_index = df_twitter.columns.index('retweet_count')
location_rdd = rdd_twitter.map(lambda row: (row[location_index], (row[id_index], row[retweet_count_index]))).groupByKey().mapValues(lambda tweet: sorted(tweet, key=lambda x: x[1], reverse=True)[:3])
location_rdd.collect()

Out[30]: [('India',
  [('1252332114948874240', 9988),
   ('1252252336921206787', 9976),
   ('1252254519116746754', 9973)]),
 ('Pakistan',
  [('1252334264248606720', 9988),
   ('1252251912084357121', 9975),
   ('1252252126694309888', 9973)]),
 ('USA',
  [('1252331777806524416', 9994),
   ('1252254239805579264', 9987),
   ('1252335464750735362', 9982)]),
 ('Italy',
  [('1252252106750377996', 9994),
   ('1252251206027816960', 9984),
   ('1252330500670337024', 9971)]),
 ('Canada',
  [('1252335430323888128', 9997),
   ('1252254877939531776', 9992),
   ('1252252082825986051', 9987)]),
 ('China',
  [('1252335780707684352', 9998),
   ('1252253596516843520', 9993),
   ('1252255562525560832', 9984)]),
 ('Chile',
  [('1252253612140490759', 9988),
   ('1252334891951427585', 9984),
   ('1252253710182481920', 9978)]),
 ('UK',
  [('1252333018578145280', 9991),
   ('1252252091822870529', 9989),
   ('1252254043973603329', 9985)]),
 ('Mexico',
  [('1252253843145912320', 9998),
   ('1252255209776189442',

In [0]:
# Using DataFrame
from pyspark.sql.window import Window
from pyspark.sql.functions import rank

df_twitter.select('id','location','retweet_count', rank().over(Window().partitionBy("location").orderBy(col("retweet_count").desc())).alias('rank')).sort("retweet_count", ascending=False).filter('rank <= 3').sort("location", "rank").show()

+-------------------+--------+-------------+----+
|                 id|location|retweet_count|rank|
+-------------------+--------+-------------+----+
|1252335430323888128|  Canada|         9997|   1|
|1252254877939531776|  Canada|         9992|   2|
|1252252082825986051|  Canada|         9987|   3|
|1252253612140490759|   Chile|         9988|   1|
|1252334891951427585|   Chile|         9984|   2|
|1252253710182481920|   Chile|         9978|   3|
|1252335780707684352|   China|         9998|   1|
|1252253596516843520|   China|         9993|   2|
|1252255562525560832|   China|         9984|   3|
|1252334028092399622| Germany|         9999|   1|
|1252330902325248000| Germany|         9997|   2|
|1252252295510855682| Germany|         9990|   3|
|1252332114948874240|   India|         9988|   1|
|1252252336921206787|   India|         9976|   2|
|1252254519116746754|   India|         9973|   3|
|1252252106750377996|   Italy|         9994|   1|
|1252251206027816960|   Italy|         9984|   2|


In [0]:
# Using spark.sql and the temporay table.
spark.sql('SELECT location, id, retweet_count FROM (SELECT *, RANK() OVER (PARTITION BY location ORDER BY retweet_count DESC) as rank FROM twitter) WHERE rank <= 3 ORDER BY location, rank ASC').show()

+--------+-------------------+-------------+
|location|                 id|retweet_count|
+--------+-------------------+-------------+
|  Canada|1252335430323888128|         9997|
|  Canada|1252254877939531776|         9992|
|  Canada|1252252082825986051|         9987|
|   Chile|1252253612140490759|         9988|
|   Chile|1252334891951427585|         9984|
|   Chile|1252253710182481920|         9978|
|   China|1252335780707684352|         9998|
|   China|1252253596516843520|         9993|
|   China|1252255562525560832|         9984|
| Germany|1252334028092399622|         9999|
| Germany|1252330902325248000|         9997|
| Germany|1252252295510855682|         9990|
|   India|1252332114948874240|         9988|
|   India|1252252336921206787|         9976|
|   India|1252254519116746754|         9973|
|   Italy|1252252106750377996|         9994|
|   Italy|1252251206027816960|         9984|
|   Italy|1252330500670337024|         9971|
|  Mexico|1252253843145912320|         9998|
|  Mexico|

#### 2.9 Total number of tweets per country

In [0]:
# Using RDD (3 points)
location_index = df_twitter.columns.index('location')
rdd_twitter.map(lambda row: row[location_index]).map(lambda location: (location, 1)).reduceByKey(lambda x, y: x + y).collect()

Out[33]: [('India', 1480),
 ('Pakistan', 1470),
 ('USA', 1539),
 ('Italy', 1422),
 ('Canada', 1441),
 ('China', 1457),
 ('Chile', 1410),
 ('UK', 1376),
 ('Mexico', 1409),
 ('Spain', 1464),
 ('Germany', 1426)]

In [0]:
# Using DataFrame (2 points)
df_twitter.groupBy("location").count().show()

+--------+-----+
|location|count|
+--------+-----+
| Germany| 1426|
|   India| 1480|
|   China| 1457|
|   Chile| 1410|
|   Italy| 1422|
|   Spain| 1464|
|     USA| 1539|
|  Mexico| 1409|
|      UK| 1376|
|  Canada| 1441|
|Pakistan| 1470|
+--------+-----+



In [0]:
# Using spark.sql and the temporay table. (1 point)
spark.sql('SELECT location, COUNT(id) FROM twitter GROUP BY location').show()

+--------+---------+
|location|count(id)|
+--------+---------+
| Germany|     1426|
|   India|     1480|
|   China|     1457|
|   Chile|     1410|
|   Italy|     1422|
|   Spain|     1464|
|     USA|     1539|
|  Mexico|     1409|
|      UK|     1376|
|  Canada|     1441|
|Pakistan|     1470|
+--------+---------+



## 3. Save Data

#### 3.1 save the data such that you have seperate folder per country (2 points)

In [0]:
# Using 
df_twitter = df_twitter.withColumn("hashtags", col("hashtags").cast("string"))
df_twitter.write.partitionBy('location').csv("location", header = 'true')

---------------------------------------------------------------------------
AnalysisException                         Traceback (most recent call last)
File <command-4093460093223405>:3
      1 # Using 
      2 df_twitter = df_twitter.withColumn("hashtags", col("hashtags").cast("string"))
----> 3 df_twitter.write.partitionBy('location').csv("location", header = 'true')

File /databricks/spark/python/pyspark/instrumentation_utils.py:48, in _wrap_function.<locals>.wrapper(*args, **kwargs)
     46 start = time.perf_counter()
     47 try:
---> 48     res = func(*args, **kwargs)
     49     logger.log_success(
     50         module_name, class_name, function_name, time.perf_counter() - start, signature
     51     )
     52     return res

File /databricks/spark/python/pyspark/sql/readwriter.py:1798, in DataFrameWriter.csv(self, path, mode, compression, sep, quote, escape, header, nullValue, escapeQuotes, quoteAll, dateFormat, timestampFormat, ignoreLeadingWhiteSpace, ignoreTrailingWhiteSp

In [0]:
df = spark.read.csv("dbfs:/location", header=True)
df.show()

+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+---------------+-------------+--------------------+-------------------+--------+
|          created_at|favorite_count|            hashtags|                 id|in_reply_to_status_id|in_reply_to_user_id_str|reply_count|retweet_count|              source|             id_str|followers_count|friends_count|     user_created_at|   extracted_source|location|
+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+---------------+-------------+--------------------+-------------------+--------+
|Mon Apr 20 15:14:...|          9038|          [COVID_19]|1252254259728355329|                 null|                   null|       6063|         7732|<a href="http://t...| 934058010875

#### 3.2 Save the data as parquet files (1 points)

In [0]:
# Using DataFrame
df_twitter.write.partitionBy('location').parquet("parquet_location")

In [0]:
df2 = spark.read.parquet("dbfs:/parquet_location", header=True)
df2.show()

+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+---------------+-------------+--------------------+-------------------+--------+
|          created_at|favorite_count|            hashtags|                 id|in_reply_to_status_id|in_reply_to_user_id_str|reply_count|retweet_count|              source|             id_str|followers_count|friends_count|     user_created_at|   extracted_source|location|
+--------------------+--------------+--------------------+-------------------+---------------------+-----------------------+-----------+-------------+--------------------+-------------------+---------------+-------------+--------------------+-------------------+--------+
|Mon Apr 20 15:14:...|          9038|          [COVID_19]|1252254259728355329|                 null|                   null|       6063|         7732|<a href="http://t...| 934058010875